<h2> 1. Data Modelling using Linear Regression Model

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [3]:
pd.set_option('display.max_columns', None)

#### Reading in the train (df_dummies.csv) and test (test_df_dummies.csv) files which we cleaned in our data cleaning python notebook.

In [4]:
df = pd.read_csv('df_dummies.csv')
test_df = pd.read_csv('test_df_dummies.csv')

In [5]:
df

,id,tranc_yearmonth,town,flat_type,flat_model,lease_commence_date,resale_price,tranc_year,tranc_month,lower,upper,mid,full_flat_type,floor_area_sqft,price_per_sqft,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_flat,2room_flat,3room_flat,4room_flat,5room_flat,exec_flat,multigen_flat,studio_apartment_flat,1room_rental,2room_rental,3room_rental,other_room_rental,latitude,longitude,mall_nearest_distance,mall_within_500m,mall_within_1km,mall_within_2km,hawker_nearest_distance,hawker_within_500m,hawker_within_1km,hawker_within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude,age_sold,town_BEDOK,town_BISHAN,town_BUKIT BATOK,town_BUKIT MERAH,town_BUKIT PANJANG,town_BUKIT TIMAH,town_CENTRAL AREA,town_CHOA CHU KANG,town_CLEMENTI,town_GEYLANG,town_HOUGANG,town_JURONG EAST,town_JURONG WEST,town_KALLANG/WHAMPOA,town_MARINE PARADE,town_PASIR RIS,town_PUNGGOL,town_QUEENSTOWN,town_SEMBAWANG,town_SENGKANG,town_SERANGOON,town_TAMPINES,town_TOA PAYOH,town_WOODLANDS,town_YISHUN
0,88471,2016-05-01,KALLANG/WHAMPOA,4 ROOM,Model A,2006,680000.0,2016,5,10,12,11,4 ROOM Model A,968.760,701.928238,25,2005,Y,N,N,N,N,142,0,0,0,96,46,0,0,0,0,0,0,0,1.314299,103.872828,1094.090418,0.0,0.0,7.0,154.753357,1.0,3.0,13.0,84,60,330.083069,Kallang,0,0,1.311540,103.871731,29.427395,1.314433,103.872600,1138.633422,Geylang Methodist School,78,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,0,1.317659,103.882504,10,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,122598,2012-07-01,BISHAN,5 ROOM,Improved,1987,665000.0,2012,7,7,9,8,5 ROOM Improved,1399.320,475.230826,9,1987,Y,N,N,N,N,112,0,0,0,56,56,0,0,0,0,0,0,0,1.346086,103.855078,866.9414477,0.0,1.0,3.0,640.151925,0.0,1.0,7.0,80,77,903.659703,Bishan,1,1,1.350580,103.848305,58.207761,1.345659,103.855381,415.607357,Kuo Chuan Presbyterian Primary School,45,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892,25,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,170897,2013-07-01,BUKIT BATOK,EXECUTIVE,Apartment,1997,838000.0,2013,7,13,15,14,EXECUTIVE Apartment,1550.016,540.639580,16,1996,Y,N,N,N,N,90,0,0,0,0,30,60,0,0,0,0,0,0,1.343867,103.760535,1459.579948,0.0,0.0,4.0,1762.082341,0.0,0.0,1.0,84,95,1334.251197,Bukit Batok,1,0,1.349561,103.749970,214.747860,1.344064,103.758613,498.849039,Keming Primary School,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013,16,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,86070,2012-04-01,BISHAN,4 ROOM,Model A,1992,550000.0,2012,4,1,5,3,4 ROOM Model A,1108.692,496.080066,11,1990,Y,Y,N,N,N,75,0,0,0,68,7,0,0,0,0,0,0,0,1.358245,103.845504,950.1751993,0.0,1.0,4.0,726.215262,0.0,1.0,9.0,32,86,907.453484,Bishan,1,1,1.350580,103.848305,43.396521,1.358045,103.845169,389.515528,Catholic High School,20,1,1.354789,103.844934,389.515528,Catholic High School,253,1,1.354789,103.844934,20,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,153632,2017-12-01,YISHUN,4 ROOM,Simplified,1987,298000.0,2017,12,1,3,2,4 ROOM Simplified,893.412,333.552717,4,1987,Y,N,N,N,N,48,0,0,0,30,18,0,0,0,0,0,0,0,1.414745,103.835532,729.7718948,0.0,1.0,2.0,1540.151439,0.0,0.0,1.0,45,0,412.343032,Khatib,0,0,1.4

#### Assigning features we plan to use for our model to 'feature_cols' variable.

In [6]:
feature_cols = ['floor_area_sqft','age_sold','mid','hawker_within_2km','mrt_nearest_distance',
                'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK',
                'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH',
                'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI',
                'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST',
                'town_KALLANG/WHAMPOA', 'town_MARINE PARADE', 'town_PASIR RIS',
                'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG',
                'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS',
                'town_YISHUN']

#### Store the model training process as a function for easier usage when we explore a different combination of features. We will refer to the RMSE and R2 scores to optimize our model.

In [ ]:
def train_test_rmse(dataframe, feature_cols):
    # Assign features and resale price from training data to X and y, respectively
    X = dataframe[feature_cols]
    y = dataframe.resale_price

    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=99)

    # Train our model using training data
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)

    # Calculate predictions using our model
    y_pred = linreg.predict(X_test)
    return X_train, X_test, y_train, y_test, y_pred

X_train, X_test, y_train, y_test, y_pred = train_test_rmse(df,feature_cols)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 54852, R2 score: 0.853


In [ ]:
# Calculating null RMSE as the baseline model
y_null = np.zeros_like(y_test, dtype=float)
y_null.fill(y_train.mean())

print(f'Null RMSE: {metrics.root_mean_squared_error(y_test,y_null):.0f}')

Null RMSE: 142924


#### Our initial model has a much lower RMSE score, 54852, than the null RMSE, 142924 which is a positive sign but can be improved.
#### An RMSE score of 54852 means that the performance of the model is able to predict prices up to an error of +/- SGD 54.852K, which is not ideal enough for agents to be used as reference due to the large variance.

In [ ]:
# Testing out by removing towns with lower correlation to resale_price

feature_cols_1 = ['floor_area_sqft','age_sold','mid','hawker_within_2km','mrt_nearest_distance',
                'town_BISHAN', 'town_BUKIT BATOK',
                'town_BUKIT MERAH', 'town_BUKIT TIMAH','town_QUEENSTOWN',
                'town_CENTRAL AREA', 'town_JURONG WEST','town_PASIR RIS', 'town_WOODLANDS','town_YISHUN']

X_train, X_test, y_train, y_test, y_pred = train_test_rmse(df,feature_cols_1)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 64225, R2 score: 0.798


In [ ]:
# Added back removed towns and changed hawker_within_2km to hawker_nearest_distance
feature_cols_2 = ['floor_area_sqft','age_sold','mid','hawker_nearest_distance','mrt_nearest_distance',
                'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK',
                'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH',
                'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI',
                'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST',
                'town_KALLANG/WHAMPOA', 'town_MARINE PARADE', 'town_PASIR RIS',
                'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG',
                'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS',
                'town_YISHUN']

X_train, X_test, y_train, y_test, y_pred = train_test_rmse(df,feature_cols_2)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 54808, R2 score: 0.853


In [ ]:
# Added in new numerical features- vacancy and cutoff_point from feature_cols_2
feature_cols_3 = ['floor_area_sqft','age_sold','mid','hawker_nearest_distance','mrt_nearest_distance', 'vacancy', 'cutoff_point',
                'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK',
                'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH',
                'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI',
                'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST',
                'town_KALLANG/WHAMPOA', 'town_MARINE PARADE', 'town_PASIR RIS',
                'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG',
                'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS',
                'town_YISHUN']

X_train, X_test, y_train, y_test, y_pred = train_test_rmse(df,feature_cols_3)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 54748, R2 score: 0.853


In [ ]:
# Added in tranc_year from feature_cols_2 and removed vacancy and cutoff point
feature_cols_4 = ['floor_area_sqft','age_sold','mid','hawker_nearest_distance','mrt_nearest_distance', 'tranc_year',
                'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK',
                'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH',
                'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI',
                'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST',
                'town_KALLANG/WHAMPOA', 'town_MARINE PARADE', 'town_PASIR RIS',
                'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG',
                'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS',
                'town_YISHUN']

X_train, X_test, y_train, y_test, y_pred = train_test_rmse(df,feature_cols_4)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 54762, R2 score: 0.853


#### After trying a few different combinations, we decided to use feature_cols_4 as our final combination of features and explore other regression models.

<h2> 2. Data Modelling using Decision Tree Regressor Model

In [50]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# Assign features and resale price from training data to X and y, respectively
X = df[feature_cols_4]
y = df.resale_price

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=99)

# Train our model using training data
regressor = DecisionTreeRegressor(random_state=99)
regressor.fit(X_train, y_train)

# Calculate predictions using our model
y_pred = regressor.predict(X_test)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 35332, R2 score: 0.939


In [ ]:
# Re-train the model with full X and y dataset
regressor.fit(X, y)

# Assign features and resale price from test data to X and y, respectively
X = test_df[feature_cols_4]
y = test_df.resale_price

# Calculate predictions using our model
y_pred = regressor.predict(X)

print (f'RMSE score: {metrics.root_mean_squared_error(y, y_pred):.0f}, R2 score: {metrics.r2_score(y, y_pred):.3f}')

RMSE score: 33916, R2 score: 0.944


#### When using the same features and training data as the Linear Regression Model, we are able to obtained a much better performance (lower RMSE, higher R2 score) when using the Decision Tree Model.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define the parameter grid to search
param_grid = {'max_depth': [3, 5, 7, None],  
              'min_samples_split': list(range(2,20)),
              'min_samples_leaf': [1, 2, 4, 8]}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=DecisionTreeRegressor(random_state=99),
                           param_grid=param_grid,
                           cv=2, 
                           scoring='neg_mean_squared_error', 
                           n_jobs=-1, 
                           verbose=1)

# Use Grid Search to find the best parameters for our model
grid_search.fit(X_train, y_train)

# Obtain optimized parameters
print("\nBest Hyperparameters found by GridSearchCV:")
print(grid_search.best_params_)

# Evaluate the best model
y_pred_tuned = grid_search.best_estimator_.predict(X_test)

print(f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred_tuned):.0f}')
print(f'R2 score: {metrics.r2_score(y_test, y_pred_tuned):.3f}')

Fitting 2 folds for each of 288 candidates, totalling 576 fits


c:\Users\brage\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,



Best Hyperparameters found by GridSearchCV:
{'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 16}
RMSE score: 32691
R2 score: 0.948


#### Using a GridSearchCV function, we are able to fine-tune the model's hyperparameters to obtain even better performance.

In [ ]:
# Re-train the model with full X and y dataset
X = df[feature_cols_4]
y = df.resale_price
grid_search.fit(X, y)

# Assign features and resale price from test data to X and y, respectively
X = test_df[feature_cols_4]
y = test_df.resale_price

# Calculate predictions using our optimized model
y_pred_tuned = grid_search.best_estimator_.predict(X)

print(f'RMSE score: {metrics.root_mean_squared_error(y, y_pred_tuned):.0f}')
print(f'R2 score: {metrics.r2_score(y, y_pred_tuned):.3f}')

Fitting 2 folds for each of 288 candidates, totalling 576 fits


c:\Users\brage\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RMSE score: 31189
R2 score: 0.953


<h2> 3. Data Modelling using Random Forest Regressor Model

In [55]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Assign features and resale price from training data to X and y, respectively
X = df[feature_cols_4]
y = df.resale_price

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=99)

# Train our model using training data
rf_regressor = RandomForestRegressor(random_state=99)
rf_regressor.fit(X_train, y_train)

# Calculate predictions using our model
y_pred = rf_regressor.predict(X_test)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 26157, R2 score: 0.967


In [ ]:
# Re-train the model with full X and y dataset
rf_regressor.fit(X, y)

# Assign features and resale price from test data to X and y, respectively
X = test_df[feature_cols_4]
y = test_df.resale_price

# Calculate predictions using our model
y_pred = rf_regressor.predict(X)

print (f'RMSE score: {metrics.root_mean_squared_error(y, y_pred):.0f}, R2 score: {metrics.r2_score(y, y_pred):.3f}')

RMSE score: 25818, R2 score: 0.968


#### After experimenting with 3 models, we decided on <b>Random Forest Regressor as our chosen model</b>. This is because it produced the lowest RMSE and highest R2 score when used to predict the resale prices of the testing data.

In [ ]:
# Assign features and resale price from training data to X and y, respectively
X = df[feature_cols_4]
y = df.resale_price

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=99)

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100],
    'max_depth': [3, 5, 7, None], 
    'min_samples_split': list(range(2,20)),
    'min_samples_leaf': [1, 2, 4, 8]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=99),
                           param_grid=param_grid,
                           cv=2, 
                           scoring='neg_mean_squared_error', 
                           n_jobs=-1, 
                           verbose=1)

# Use Grid Search to find the best parameters for our model
grid_search.fit(X_train, y_train)

# Obtain optimized parameters
print("\nBest Hyperparameters found by GridSearchCV:")
print(grid_search.best_params_)

# Evaluate the best model
y_pred_tuned = grid_search.best_estimator_.predict(X_test)

print(metrics.root_mean_squared_error(y_test, y_pred_tuned))
print(metrics.r2_score(y_test, y_pred_tuned))

Fitting 2 folds for each of 288 candidates, totalling 576 fits


c:\Users\brage\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,



Best Hyperparameters found by GridSearchCV:
{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 100}
26103.469249794147
0.9666431176595197


In [ ]:
# Re-train the model with full X and y dataset
X = df[feature_cols_4]
y = df.resale_price
grid_search.fit(X, y)

# # Evaluate the best model
X = test_df[feature_cols_4]
y = test_df.resale_price

y_pred_tuned = grid_search.best_estimator_.predict(X)

print(f'RMSE score: {metrics.root_mean_squared_error(y, y_pred_tuned):.0f}')
print(f'R2 score: {metrics.r2_score(y, y_pred_tuned):.3f}')

RMSE score: 25699
R2 score: 0.968


#### Only minor improvements to performance when we use GridSearchCV to tune our model's hyperparameters.

In [106]:
kaggle_submission = test_df['id']
kaggle_submission = pd.DataFrame(kaggle_submission)
kaggle_submission['Predicted'] = y_pred_tuned
kaggle_submission.to_csv('kaggle_submission.csv', index = False)

#### Export the dataframe containing id and predicted resale prices using our model and exported it out as a CSV file.
#### This is to be used for our kaggle submission to evaluate the score.